#### Dynamic model creation

There are some occasions where the shape of a model is not known until runtime. For this pydantic provides the `create_model` method to allow models to be created on the fly.

In [1]:
from pydantic import BaseModel, create_model, ValidationError, validator

In [2]:
DynamicFoobarModel = create_model("DynamicFoobarModel", foo=(str, ...), bar=123)

In [3]:
class StaticFoobarModel(BaseModel):
    foo: str
    bar: int = 123

Here `StaticFoobarModel` and `DynamicFoobarModel` are identical. Fields are defined by either a tuple of the form `(<type>, <default value>)` or just a default value. The special key word arguments `__config__` and `__base__` can be used to customise the new model. This includes extending a base model with extra fields.

In [4]:
class FooModel(BaseModel):
    foo: str
    bar: int = 123

In [5]:
BarModel = create_model(
    "BarModel",
    apple="russet",
    banana="yellow",
    __base__=FooModel,
)

In [6]:
print(BarModel)
print(BarModel.__fields__.keys())

<class 'pydantic.main.BarModel'>
dict_keys(['foo', 'bar', 'apple', 'banana'])


You can also add validators by passing a dict to the `__validators__` argument.

In [7]:
def username_alphanumeric(cls, v):
    assert v.isalnum(), "must be alphanumeric"
    return v

In [8]:
validators = {
    "username_validator": validator("username")(username_alphanumeric)
}

In [9]:
UserModel = create_model(
    "UserModel",
    username=(str, ...),
    __validators__=validators
)

In [10]:
user = UserModel(username="scolvin")
print(user)

username='scolvin'


In [11]:
try:
    print(UserModel(username="scolvi%n"))
except ValidationError as e:
    print(e)

1 validation error for UserModel
username
  must be alphanumeric (type=assertion_error)
